# Pré-traitement pour l'entraînement

In [1]:
import numpy as np
import pandas as pd

from utils.preprocessing.label import TopicLDA
from utils.preprocessing.augmentation import (
    BeyondBackTranslator, GenderSwap)

Chargement des données

In [2]:
data_path = "../data/"
with open(data_path + 'categories_string.csv') as f:
    categories = dict(map(lambda x:x.strip('\n').split(',')[::-1], f.readlines()[1:]))
    
X = pd.read_json(data_path + 'train.json').set_index('Id')
y = pd.read_csv(data_path + 'train_label.csv', index_col='Id', dtype={'Category': 'category'}).Category

## Topic LDA
Nettoyage des labels.  
Pour rappel, le nettoyage a été effectué pour le label "architect" qui comporte à la fois des données pour les architectes de constructions proches du label "interior designer" et les architect informatiques proches du label "software engineer".

La classe `TopicLDA` que nous avons implémenté possède les paramètres optimales pour le label "architect" par défaut. Nous avons choisi de ne pas étendre le nettoyage des labels sur l'ensemble des données puisque l'opération d'optimisation des paramètres est coûteuse en temps et surtout parce que nous n'avons pas observé d'ambiguités dans les autres labels.

In [3]:
X_arch = X[y == '24']
categories['24']

'architect'

Clusterisation des descriptions en 2 groupes *(paramètre optimal pour 'architect')*

In [4]:
clust = TopicLDA(2)
logits_arch = clust.fit_predict(X_arch.description)
logits_arch

array([[0.35061345, 0.6493866 ],
       [0.96516657, 0.03483342],
       [0.85230094, 0.14769909],
       ...,
       [0.07843395, 0.921566  ],
       [0.03581987, 0.9641802 ],
       [0.9616145 , 0.03838551]], dtype=float32)

Le label '24' sera gardé pour le cluster 0 et le label '29' sera créé pour le cluster 1.  
Lors de la classification, une couche `Lambda` re-calibrera les prédictions en additionnant les logits de ceux deux labels.

In [5]:
y_arch = np.vectorize({0:'24',1:'29'}.get)(np.argmax(logits_arch, axis = 1))
y_new = y.cat.add_categories('29')
y_new[y_new == '24'] = y_arch

Exemple

In [6]:
dict(zip(y_new[3:5], X_arch.description.to_list()[2:4]))

{'29': " He holds a Master of Science degree in Computer Science from California State University, San Bernardino. His focus was on online banking applications. He also has a bachelor's degree from Wuhan University, China. Xinsheng was a QA engineer at VMware, Inc. He later led a team in developing four educational computer games for the Escambia County School District, Florida. He has worked on Geographical Information Systems (GIS). Xinsheng has rich experience in J2EE technologies. He has extensive experience in content management systems (CMS) including Alfreso. He is an expert in Web Portal technologies. Xinsheng has hands-on experience in eight Liferay Portal projects.",
 '24': ' Following studies at Kyoto University Graduate School, Hirata worked for architect Toyo Ito & Associates (1997-2005), before establishing his own studio in 2005.'}

## Beyond Back translation
Cette procédure consiste à traduire le texte dans une langue et la re-traduire dans sa langue d'origine.  
Le but est de créer des nuances de mots dans les phrases en profitant des différences d'expressions entre les langues.

Dans cet exemple, nous utiliserons le français comme langue intermédiaire.

In [7]:
bbt = BeyondBackTranslator("en", "fr")

La méthode `.compute` permet de connaître la classe sous-représentée pour chaque label et renvoie les statistiques associées.  
Les 5 premiers éléments de sortie.

In [8]:
stats = bbt.compute(X.description, X.gender, y)
stats[:5]

[{'Category': '0', 0: 360, 1: 1137, 'low': 'F'},
 {'Category': '1', 0: 3398, 1: 717, 'low': 'M'},
 {'Category': '10', 0: 125, 1: 706, 'low': 'F'},
 {'Category': '11', 0: 4581, 1: 7026, 'low': 'F'},
 {'Category': '12', 0: 345, 1: 1294, 'low': 'F'}]

La méthode `.oversampling` choisi aléatoirement les données pour le sur-échantillonnage en suivant la règle suivante pour chaque label de manière générale:

- La moitié des données de la classe sous-représentée
- Le quart des données de la classe dominante à inverser

Cette règle change si le sur-échantillonnage recrée crée le déséquilibrement de labels presque équilibrés.

In [9]:
bbt.oversampling(X.description, X.gender, y)
f"{bbt.oversampled.shape[0]:,} données sélectionnées pour la BBT"

'62,589 données sélectionnées pour la BBT'

La méthode `.generate` est la fonction à utiliser.  
Elle effectue les 2 tâches précédentes et effectue la traduction.

Exemple:
> Ne pas lancer le code suivant, couteux en temps

In [10]:
bbt.generate(
    X.description, X.gender, y,
    swap = True
)

Jane is the pastor's wife and she gets up every morning to light the candles in the church
Jane is the priest's spouse and she wakes up every morning to light the candles of the chapel


## Gender Swap
Le `Gender Swap` consiste à intervertir le genre des mots présents dans la phrase s'il existe et si la similarité cosinus de l'inverse est supérieur au seuil.  
Les données à inverser provenant de la Beyond Back Translation doivent passer par le Gender Swapping.

Pour la démonstration, nous utilisons le modèle `word2vec-google-news-300`.
> Ne pas lancer le code suivant, couteux en temps

In [11]:
swaper = GenderSwap({'F': 'female', 'M': 'male'}, 'word2vec-google-news-300')

La méthode `.sentswap` effectue l'interversion

In [12]:
swaper.sentswap(
    sentence = "Jane is the priest's spouse and she wakes up every morning to light the candles of the chapel",
    source = "female", target = "male", thres = .65
)

"Sally is the priest 's husband and he wakes up every afternoon to light the candle of the chapel"

Comme `BeyondBackTranslator`, la méthode `.generate` de `GenderSwap` gère le swapping pour les données qui sortent de la BBT

In [13]:
swaper.generate(
    X.description, X.gender,
    thres = .65
)

# Resultats
Pendant l'entraînement, après avoir divisé nos données en 3 échantillons (`train`, `validation`, `test`) nous effectuons l'augmentation sur l'échantillon `train`.